In [1]:
from ltlcross_wrapper import ResAnalyzer
import pandas as pd

In [2]:
from tools import tools, benchmark_names

Visualization of the cumulative data over all benchmarks

In [60]:
def gather_cumulative(benchmarks, tools=None, transpose=True, **kwargs):
    data = pd.DataFrame()
    for (name,b) in benchmarks.items():
        tmp = pd.DataFrame(b.cumulative(tool_set=tools, highlight=False, **kwargs))
        tmp.columns = [name]
        data = data.append(tmp.transpose())
    if transpose:
        return data.transpose().style.apply(highlight_min, axis=0)
    else:
        return data.style.apply(highlight_min, axis=1)
    
def gather_mins(benchmarks, tools=None, transpose=True):
    data = pd.DataFrame()
    for (name,b) in benchmarks.items():
        tmp = b.min_counts(tool_set=tools)
        tmp.columns = pd.MultiIndex.from_tuples([(name, c) for c in tmp.columns])
        data = data.append(tmp.transpose(),sort=False).fillna(0)
    if transpose:
        return data.transpose().style.apply(highlight_max, axis=0)
    return data.style.apply(highlight_max, axis=1)

def highlight_min(s):    
    is_min = s == s.min()
    return ['background-color: lightgreen' if v else '' for v in is_min]

def highlight_max(s):    
    is_max = s == s.max()
    return ['background-color: lightgreen' if v else '' for v in is_max]

In [61]:
names = benchmark_names
tools = ["yes.autfilt","yes.ncsb#best","yes.goal#fri","yes.goal#pit","yes.buechenic"]

In [62]:
benchmarks = {}
for name in names:
    b = ResAnalyzer(f"data/{name}.csv", tool_set=tools, cols=["states","time","acc","transitions","edges"])
    b.name = name
    b.orig_count = len(b.values)
    b.clean_count = len(b.values.dropna())
    benchmarks[name] = b

In [63]:
gather_cumulative(benchmarks)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
yes.autfilt,611,191,188,2439,2892,5492
yes.buechenic,1388,866,309,3638,5766,6330
yes.goal#fri,625,322,189,2490,3367,5305
yes.goal#pit,615,872,243,2451,3420,7645
yes.ncsb#best,622,240,186,2473,2828,5003


## Minimal automata
The follwing table shows for how many formulas each tool produces automaton that has the smallest number of states. The minimum ranges over the considered tools. The number in `min hits` shows how many times the same size as the smallest automaton was achieved. The number in `unique min hits` counts only cases where the given tool is the only tool with such a small automaton.

In [64]:
gather_mins(benchmarks)

### Time in seconds

In [65]:
gather_cumulative(benchmarks, col="time")

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
yes.autfilt,8.57435,2.5377,14.1334,25.9194,28.1235,155.879
yes.buechenic,980.031,239.804,672.446,1136.08,1240.11,3130.46
yes.goal#fri,862.555,420.565,332.866,2347.04,2483.33,3709.36
yes.goal#pit,791.033,478.867,334.117,2364.22,2570.98,4112.3
yes.ncsb#best,10.2817,2.85127,2.68155,29.3337,30.7912,384.036


## Closer look at benchmark by input automaton type

In [57]:
for t in ["nd", "sd", "det"]:
    bench = {n: b for n, b in benchmarks.items() if n[-3:].find(t) >= 0}
    display(t, gather_cumulative(bench, tools, transpose=True))
    for b in bench.values():
        display(b.cross_compare(tools, include_fails=True))
    print("\n\n\n")

'nd'

,literature_nd,random_nd
tool,,
yes.autfilt,188,5492
yes.buechenic,309,6330
yes.goal#fri,189,5305
yes.goal#pit,243,7645
yes.ncsb#best,186,5003


,yes.autfilt,yes.ncsb#best,yes.goal#fri,yes.goal#pit,yes.buechenic,V
yes.autfilt,nan,7,5,11,18,41
yes.ncsb#best,12,nan,6,15,18,51
yes.goal#fri,12,9,nan,15,19,55
yes.goal#pit,6,2,1,nan,16,25
yes.buechenic,2,2,1,4,nan,9


,yes.autfilt,yes.ncsb#best,yes.goal#fri,yes.goal#pit,yes.buechenic,V
yes.autfilt,nan,195,223,357,374,1149
yes.ncsb#best,253,nan,254,374,407,1288
yes.goal#fri,230,213,nan,368,391,1202
yes.goal#pit,100,86,88,nan,281,555
yes.buechenic,125,91,109,218,nan,543


'sd'

...

,yes.autfilt,yes.ncsb#best,yes.goal#fri,yes.goal#pit,yes.buechenic,V
yes.autfilt,nan,108,237,229,483,1057
yes.ncsb#best,142,nan,246,234,486,1108
yes.goal#fri,109,91,nan,151,473,824
yes.goal#pit,69,54,148,nan,451,722
yes.buechenic,13,14,24,48,nan,99


'det'

,literature_det,random_det
tool,,
yes.autfilt,611,2439
yes.buechenic,1388,3638
yes.goal#fri,625,2490
yes.goal#pit,615,2451
yes.ncsb#best,622,2473


,yes.autfilt,yes.ncsb#best,yes.goal#fri,yes.goal#pit,yes.buechenic,V
yes.autfilt,nan,14,13,10,152,189
yes.ncsb#best,14,nan,8,4,148,174
yes.goal#fri,15,13,nan,5,152,185
yes.goal#pit,15,11,6,nan,152,184
yes.buechenic,0,4,0,0,nan,4


,yes.autfilt,yes.ncsb#best,yes.goal#fri,yes.goal#pit,yes.buechenic,V
yes.autfilt,nan,64,40,42,494,640
yes.ncsb#best,9,nan,30,16,489,544
yes.goal#fri,20,57,nan,36,492,605
yes.goal#pit,11,48,21,nan,492,572
yes.buechenic,0,5,2,2,nan,9


## Without simplifications of Spot
GOAL#pit runs removing dead and unreachable states, Buechenic probably does not create such states. The rest of the tools does not remove them.

We can observe that Fribourg generates large ammount of unnecessary states before simplifications.

In [67]:
no_tools = [t.replace("yes","no") for t in tools]

In [68]:
gather_cumulative(benchmarks, tools=no_tools)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
no.autfilt,613,257,205,2442,3578,7604
no.buechenic,1635,915,313,4710,6903,7627
no.goal#fri,1296,2518,2946,5059,17369,56448
no.goal#pit,634,513,336,2510,4990,13834
no.ncsb#best,804,439,562,3060,6446,20750


## Minimal automata
The follwing table shows for how many formulas each tool produces automaton that has the smallest number of states. The minimum ranges over the considered tools. The number in `min hits` shows how many times the same size as the smallest automaton was achieved. The number in `unique min hits` counts only cases where the given tool is the only tool with such a small automaton.

In [69]:
gather_mins(benchmarks, tools=no_tools)

### Time in seconds

In [70]:
gather_cumulative(benchmarks, col="time", tools=no_tools)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
no.autfilt,7.94785,2.59661,1.14557,25.8379,27.3733,32.8517
no.buechenic,1005.34,234.268,700.125,1134.28,1222.84,3201.37
no.goal#fri,857.512,421.558,315.609,2364.8,2478.42,3531.84
no.goal#pit,856.772,506.556,353.231,2342.6,2569.2,3901.2
no.ncsb#best,1.75239,0.712357,0.857351,5.64361,29.1077,15.3791


In [58]:
for t in ["nd", "sd", "det"]:
    bench = {n: b for n, b in benchmarks.items() if n[-3:].find(t) >= 0}
    display(t, gather_cumulative(bench, no_tools, transpose=True))
    for b in bench.values():
        display(b.cross_compare(no_tools))
    print("\n\n\n")

'nd'

,literature_nd,random_nd
tool,,
no.autfilt,205,7604
no.buechenic,313,7627
no.goal#fri,2946,56448
no.goal#pit,336,13834
no.ncsb#best,562,20750


,no.autfilt,no.ncsb#best,no.goal#fri,no.goal#pit,no.buechenic,V
no.autfilt,nan,19,20,17,18,74
no.ncsb#best,1,nan,17,3,6,27
no.goal#fri,0,3,nan,0,5,8
no.goal#pit,3,17,19,nan,12,51
no.buechenic,2,14,15,8,nan,39


,no.autfilt,no.ncsb#best,no.goal#fri,no.goal#pit,no.buechenic,V
no.autfilt,nan,443,500,435,364,1742
no.ncsb#best,56,nan,477,227,212,972
no.goal#fri,0,23,nan,3,26,52
no.goal#pit,61,268,494,nan,238,1061
no.buechenic,135,288,474,261,nan,1158


'sd'

...

,no.autfilt,no.ncsb#best,no.goal#fri,no.goal#pit,no.buechenic,V
no.autfilt,nan,372,500,313,471,1656
no.ncsb#best,50,nan,500,204,369,1123
no.goal#fri,0,0,nan,1,108,109
no.goal#pit,86,244,499,nan,412,1241
no.buechenic,28,131,390,88,nan,637


'det'

,literature_det,random_det
tool,,
no.autfilt,613,2442
no.buechenic,1635,4710
no.goal#fri,1296,5059
no.goal#pit,634,2510
no.ncsb#best,804,3060


,no.autfilt,no.ncsb#best,no.goal#fri,no.goal#pit,no.buechenic,V
no.autfilt,nan,141,152,25,152,470
no.ncsb#best,0,nan,150,4,144,298
no.goal#fri,0,0,nan,1,90,91
no.goal#pit,0,137,151,nan,151,439
no.buechenic,0,7,61,0,nan,68


,no.autfilt,no.ncsb#best,no.goal#fri,no.goal#pit,no.buechenic,V
no.autfilt,nan,449,500,71,500,1520
no.ncsb#best,0,nan,490,36,477,1003
no.goal#fri,0,8,nan,0,208,216
no.goal#pit,0,414,492,nan,498,1404
no.buechenic,0,22,287,2,nan,311


## Errors
We can observe several timeouts (120s) and 2 parsing errors on GOAL configurations. These occur on 2 formulas equivalent to false. GOAL does not see the automata as Büchi.

In [38]:
for name, b in benchmarks.items():
    display(name, b.get_error_counts())

'literature_det'

,timeout,parse error,incorrect,crash,no output
no.buechenic,4,0,0,0,0
no.goal#pit,1,0,0,0,0
yes.buechenic,4,0,0,0,0


'literature_sd'

,timeout,parse error,incorrect,crash,no output
no.buechenic,1,0,0,0,0
no.goal#fri,1,0,0,0,0
no.goal#pit,2,0,0,0,0
yes.buechenic,1,0,0,0,0
yes.goal#fri,1,0,0,0,0


'literature_nd'

...

,timeout,parse error,incorrect,crash,no output
no.goal#fri,0,2,0,0,0
no.goal#pit,0,2,0,0,0
yes.goal#fri,0,2,0,0,0
yes.goal#pit,0,2,0,0,0


'random_sd'

,timeout,parse error,incorrect,crash,no output
tool,,,,,


'random_nd'

,timeout,parse error,incorrect,crash,no output
no.buechenic,4,0,0,0,0
no.goal#fri,3,0,0,0,0
no.goal#pit,4,0,0,0,0
yes.autfilt,1,0,0,0,0
yes.buechenic,4,0,0,0,0
yes.goal#fri,4,0,0,0,0
yes.goal#pit,6,0,0,0,0
yes.ncsb#best,1,0,0,0,0
yes.ncsb#pldi,1,0,0,0,0
yes.ncsb#spot,1,0,0,0,0


In [16]:
b = benchmarks["random_nd"]

frib = "yes.goal#fri"
pldi = "yes.ncsb#pldi"
ncsb = "yes.ncsb#spot"
spot = "yes.autfilt"
best = "yes.ncsb#best"

In [17]:
b.bokeh_scatter_plot(frib, best, alpha=.5)

Loading BokehJS ...

Figure(id='1002', ...)

In [18]:
b.bokeh_scatter_plot(pldi, ncsb)

Loading BokehJS ...

Figure(id='1124', ...)

In [19]:
b.bokeh_scatter_plot(best, spot)

Loading BokehJS ...

Figure(id='1256', ...)

In [20]:
display(b.min_counts(), b.min_counts(unique_only=True))

,0
no.autfilt,6
no.buechenic,23
yes.autfilt,83
yes.buechenic,37
yes.goal#fri,98
yes.goal#pit,37
yes.ncsb#best,160
yes.ncsb#pldi,94
yes.ncsb#spot,150


,0
yes.autfilt,24
yes.buechenic,13
yes.goal#fri,40
yes.goal#pit,1


In [19]:
ncsb_tools = [t for t in b.tools if t.startswith("yes") and t.find("ncsb") >= 0]

In [20]:
b.compute_best(tool_set=ncsb_tools,new_col_name="NCSB-best")

In [21]:
b.bokeh_scatter_plot(frib, "NCSB-best")

Loading BokehJS ...

Figure(id='1585', ...)